<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/Optimized%20SWAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Install Autokeras library


# Import required libraries
import autokeras as ak
import timeit
import numpy as np
import tensorflow as tf
# from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Dense, concatenate, Dropout, Flatten, Activation
from keras import backend as K
from keras.utils import get_custom_objects
from keras.utils import  to_categorical, plot_model
from tensorflow.keras.optimizers import Adam
from keras.datasets import mnist


# Set random seed
np.random.seed(110)
# Defining new activation functions
def define_activation_functions():
    """
    Define custom activation functions and update the custom objects dictionary.
    """
    # Define activation functions and their respective names
    activation_functions = [
        ('X_1', lambda x: K.pow(x, 1)),
        ('X_2', lambda x: K.pow(x, 2) / 2),
        ('X_2_', lambda x: K.pow(x, 2) / 24),
        ('X_2__', lambda x: K.pow(x, 2) / 720),
        ('X_2___', lambda x: K.pow(x, 2) / 40320),
        ('X_3', lambda x: K.pow(x, 3) / 6),
        ('X_4', lambda x: K.pow(x, 4) / 24),
        ('X_5', lambda x: K.pow(x, 5) / 120),
        ('X_6', lambda x: K.pow(x, 6) / 720),
    ]

    # Update the custom objects dictionary with the defined activation functions
    for name, function in activation_functions:
        get_custom_objects().update({name: Activation(function)})

# Define and register the custom activation functions
define_activation_functions()

def create_optimized_model(input_dim,  hidden_dim , output_dim ):
    """
    Create an optimized SWAG model with a custom architecture.

    :param input_dim: int, dimension of the input data
    :param output_dim: int, dimension of the output data
    :param hidden_dim: int, hidden layer dimension, default is 50
    :return: Model, a compiled Keras model
    """

    # Define input layer
    input_layer = Input(shape=(input_dim,))

    # First layer with custom activations
    layer_1_x1 = Dense(hidden_dim, activation='X_1')(input_layer)
    layer_1_x2 = Dense(hidden_dim, activation='X_2')(input_layer)
    concat_first_layer = concatenate([layer_1_x1, layer_1_x2])

    # Second layer with custom activations
    layer_x3_x4 = Dense(hidden_dim, activation='X_2_')(concat_first_layer)

    # Third layer with custom activations
    layer_x5_x6 = Dense(hidden_dim, activation='X_2__')(layer_x3_x4)



    # Concatenate all layers
    concat_second_layer = concatenate([layer_1_x1, layer_1_x2, concat_first_layer,
                                       layer_x3_x4, layer_x5_x6])

    # Output layer for the concatenated layers
    output_first_layer = Dense(hidden_dim, activation='linear')(concat_second_layer)

    # Final output layer
    output_layer = Dense(output_dim, activation='linear')(output_first_layer)

    # Create and compile the model
    model = Model(input_layer, output_layer)
    # Define the optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
    # model.compile(loss='mean_squared_error', optimizer='adam')

    return model


optimized_model = create_optimized_model(784,784, 10)

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = (x_train + 10) / 300.0, (x_test + 10) / 300.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create and train the model
optimized_model = create_optimized_model(784, 500, 10)
batch_size = 100
epochs = 2
start = timeit.default_timer()
history = optimized_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

Epoch 1/2
600/600 [==============================] - 40s 65ms/step - loss: 0.0415 - accuracy: 0.8376 - val_loss: 0.0257 - val_accuracy: 0.9094
Epoch 2/2
600/600 [==============================] - 37s 62ms/step - loss: 0.0207 - accuracy: 0.9359 - val_loss: 0.0197 - val_accuracy: 0.9318
Training time: 83.38027213499981 seconds


In [19]:
# Load and preprocess Iris data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

# Create and train the model
optimized_model = create_optimized_model(4, 50, 3) # input_dim=4, hidden_dim=50, output_dim=3
batch_size = 10
epochs = 10
start = timeit.default_timer()
history = optimized_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

Epoch 1/10
12/12 [==============================] - 2s 45ms/step - loss: 0.2319 - accuracy: 0.6500 - val_loss: 0.1185 - val_accuracy: 0.9000
Epoch 2/10
12/12 [==============================] - 0s 7ms/step - loss: 0.1180 - accuracy: 0.8250 - val_loss: 0.0972 - val_accuracy: 0.9333
Epoch 3/10
12/12 [==============================] - 0s 6ms/step - loss: 0.0899 - accuracy: 0.9000 - val_loss: 0.0681 - val_accuracy: 0.9667
Epoch 4/10
12/12 [==============================] - 0s 10ms/step - loss: 0.0740 - accuracy: 0.9167 - val_loss: 0.0571 - val_accuracy: 0.9667
Epoch 5/10
12/12 [==============================] - 0s 8ms/step - loss: 0.0642 - accuracy: 0.9333 - val_loss: 0.0582 - val_accuracy: 1.0000
Epoch 6/10
12/12 [==============================] - 0s 16ms/step - loss: 0.0589 - accuracy: 0.9333 - val_loss: 0.0566 - val_accuracy: 1.0000
Epoch 7/10
12/12 [==============================] - 0s 7ms/step - loss: 0.0514 - accuracy: 0.9583 - val_loss: 0.0448 - val_accuracy: 1.0000
Epoch 8/10
12/12 

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import timeit
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, concatenate, Activation
from keras import backend as K
from keras.utils import get_custom_objects
from tensorflow.keras.optimizers import Adam
from sklearn import datasets




# Load Boston Housing data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the model
input_dim = X_train.shape[1]
optimized_model = create_optimized_model(input_dim, input_dim , 1) # input_dim and hidden_dim
batch_size = 10
epochs = 100
start = timeit.default_timer()
history = optimized_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")


Epoch 1/100
41/41 [==============================] - 2s 7ms/step - loss: 492.7659 - accuracy: 0.0000e+00 - val_loss: 403.6549 - val_accuracy: 0.0000e+00
Epoch 2/100
41/41 [==============================] - 0s 3ms/step - loss: 286.8319 - accuracy: 0.0000e+00 - val_loss: 224.1312 - val_accuracy: 0.0000e+00
Epoch 3/100
41/41 [==============================] - 0s 3ms/step - loss: 157.0345 - accuracy: 0.0000e+00 - val_loss: 146.3470 - val_accuracy: 0.0000e+00
Epoch 4/100
41/41 [==============================] - 0s 3ms/step - loss: 95.7073 - accuracy: 0.0000e+00 - val_loss: 89.6485 - val_accuracy: 0.0000e+00
Epoch 5/100
41/41 [==============================] - 0s 3ms/step - loss: 54.8925 - accuracy: 0.0000e+00 - val_loss: 60.0018 - val_accuracy: 0.0000e+00
Epoch 6/100
41/41 [==============================] - 0s 3ms/step - loss: 37.5024 - accuracy: 0.0000e+00 - val_loss: 46.6903 - val_accuracy: 0.0000e+00
Epoch 7/100
41/41 [==============================] - 0s 3ms/step - loss: 30.1984 - accur